# 使用元类

## 使用type()定义实例

动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义的，而是运行时动态创建的。

`type()`函数可以用来检测对象的类型:

In [1]:
class Hello(object):
    def hello(self, name='world'):
        print('Hello, %s.' % name)
        
h = Hello()
print("type(Hello): ", type(Hello))
print("type(h): ", type(h))

type(Hello):  <class 'type'>
type(h):  <class '__main__.Hello'>


上述可知, `Hello`的类型是`tyep`;`h`的类型是`class`;

`type()`函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过`type()`函数创建出`Hello`类，而无需通过`class Hello(object)...`的定义：

In [4]:
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)

Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

h = Hello() # 定义实例
print("h: ", h.hello())
print("type(Hello): ", type(Hello))
print("type(h): ", type(h))

Hello, world.
h:  None
type(Hello):  <class 'type'>
type(h):  <class '__main__.Hello'>


要创建一个`class`对象，`type()`函数依次传入3个参数：

1. class的名称；
2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

通过type()函数创建的类和直接写class是完全一样的，因为Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用type()函数创建出class。

## 使用metaclass创建class

除了使用type()动态创建类以外，要控制类的创建行为，还可以使用metaclass。metaclass，直译为元类，简单的解释就是：当我们定义了类以后，就可以根据这个类创建出实例，所以：先定义类，然后创建实例。但是如果我们想创建出类呢？那就必须根据metaclass创建出类，所以：先定义metaclass，然后创建类。

连接起来就是：先定义metaclass，就可以创建类，最后创建实例。

所以，metaclass允许你创建类或者修改类。换句话说，你可以把类看成是metaclass创建出来的“实例”。

我们先看一个简单的例子，这个metaclass可以给我们自定义的MyList增加一个add方法：

定义ListMetaclass，按照默认习惯，metaclass的类名总是以Metaclass结尾，以便清楚地表示这是一个metaclass：

In [5]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)
    
# 有了ListMetaclass，我们在定义类的时候还要指示使用ListMetaclass来定制类，
# 传入关键字参数metaclass
class MyList(list, metaclass=ListMetaclass):
    pass

L = MyList()
L.add(1)
print("L: ", L)

# 而普通的list没有add()方法：
L2 = list()
L2.add(1)

L:  [1]


AttributeError: 'list' object has no attribute 'add'

当我们传入关键字参数`metaclass`时，魔术就生效了，它指示`Python`解释器在创建`MyList`时，要通过`ListMetaclass.__new__()`来创建，在此，我们可以修改类的定义，比如，加上新的方法，然后，返回修改后的定义。

`__new__()`方法接收到的参数依次是：

1. 当前准备创建的类的对象；
2. 类的名字；
3. 类继承的父类集合；
4. 类的方法集合。